In [1]:
import os
import sys
import pandas as pd 
import numpy as np
from data_import import *
from phenotype_correlation import *
from genotype_correlation import *
from baseline_mixed_model import *
import statsmodels.api as sm

# import data
geno_df, pheno_df = import_data_svenson()

Samples:
['F326', 'F327', 'F329', 'F330', 'F331', 'F332', 'F333', 'F334', 'F335', 'F336', 'F337', 'F338', 'F339', 'F340', 'F341', 'F342', 'F343', 'F344', 'F345', 'F346', 'F347', 'F348', 'F349', 'F350', 'F351', 'F352', 'F353', 'F354', 'F355', 'F356', 'F357', 'F358', 'F359', 'F360', 'F361', 'F362', 'F363', 'F364', 'F365', 'F366', 'F367', 'F368', 'F369', 'F371', 'F372', 'F373', 'F374', 'F375', 'F376', 'F377', 'F378', 'F379', 'F380', 'F381', 'F382', 'F383', 'F384', 'F385', 'F386', 'F387', 'F388', 'F389', 'F390', 'F391', 'F392', 'F393', 'F394', 'F395', 'F396', 'F397', 'F398', 'F399', 'F400', 'F401', 'F402', 'F403', 'F405', 'F406', 'F407', 'F408', 'F410', 'F411', 'F412', 'F413', 'F414', 'F415', 'F416', 'F417', 'F418', 'F419', 'F420', 'F421', 'F422', 'F423', 'F424', 'F425', 'M326', 'M327', 'M328', 'M329', 'M330', 'M331', 'M332', 'M333', 'M334', 'M335', 'M336', 'M337', 'M339', 'M340', 'M341', 'M342', 'M343', 'M344', 'M345', 'M347', 'M348', 'M349', 'M350', 'M351', 'M352', 'M353', 'M354', 'M355'

/Users/harryyang/Documents/Research/Class/CS229/cm-229/data_import.py:75: RuntimeWarning: Mean of empty slice
  np.nanmean(geno_df_select_transpose, axis=0))/np.nanstd(geno_df_select_transpose, axis=0)
/Users/harryyang/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [215]:
# testing with ghrelin and leptin 
phenotype_2 = "weight1"
phenotype_1 = "length1"
phenotype_3 = "fat_mri"
missing_rate = 0.5
verbose = False
sample_list = list()

In [239]:
corr_mat = calculate_highly_correlated_phenotypes(pheno_df)

print("The correlation between %s and %s is %f" % (phenotype_1, phenotype_2, corr_mat[phenotype_1][phenotype_2]))

# bind phenotype into list to extract
phenotype_list = [phenotype_1, phenotype_2]

# extract the phenotypes 
geno_select, pheno_select = select_phenotype_multiple_phenotypes(geno_df, pheno_df, phenotype_list = phenotype_list, verbose = verbose)

The correlation between length1 and fat_mri is nan


In [256]:
corr_mat = calculate_highly_correlated_phenotypes(pheno_df)

print("The correlation between %s and %s is %f" % (phenotype_1, phenotype_2, corr_mat[phenotype_1][phenotype_2]))

# bind phenotype into list to extract
phenotype_list = [phenotype_1, phenotype_2]

# extract the phenotypes 
geno_select, pheno_select = select_phenotype_multiple_phenotypes(geno_df, pheno_df, phenotype_list = phenotype_list, verbose = verbose)

# separate training and test dataset 
geno_tr, pheno_tr, geno_test, pheno_test, test_sample_list = separate_training_test(geno_select, pheno_select, missing_rate = missing_rate, sample_list_select = sample_list)

# remove duplciates
geno_test_new = geno_test.loc[:,~geno_test.columns.duplicated()]
geno_test = geno_test_new[pheno_test[phenotype_2].index]

The correlation between length1 and fat_mri is nan


In [257]:
# saving below 
# # perform simple ridge to identify the top SNPs 
# lm_ridge = sm.OLS(endog = pheno_tr[phenotype_2], exog = geno_tr.transpose()).fit_regularized(L1_wt = 1.0)

# if verbose: 
#     print(lm_ridge.params)

# # select top SNPs with highest effect size for select run
# top_N_idx = np.argsort(abs(lm_ridge.params))[-top_N:]

# if verbose:
# 	top_N_values = [lm_re.params[i] for i in top_N_idx]
# 	print(top_N_values)

# top_N_snps = geno_tr.iloc[top_N_idx].index

# sklearn test 

# clf = Ridge(alpha = 1.0)
# a = clf.fit(y = pheno_tr[phenotype_2], X = geno_tr.transpose())

# # select top N 
# top_N = 10
# top_N_idx = np.argsort(abs(a.coef_))[-top_N:]

# print (top_N_idx)

# top_N_values = [a.coef_[i] for i in top_N_idx]
# print (top_N_values)

# top_N_snps = geno_tr.iloc[top_N_idx].index
# print(top_N_snps)


# perform OLS 
lm = sm.OLS(endog = pheno_tr[phenotype_2], exog = pheno_tr[phenotype_1]).fit()

if verbose:	
    print("The linear model summary for predicting phenotype %a based on phenotype %a" % (phenotype_2, phenotype_1))
    print(lm.summary())	
    print(lm.params)	

# prediction for fixed effect
predictions_fe = lm.predict(pheno_test[phenotype_1])

# perform ridge regression on the residual (random effect part)
residuals = pheno_tr[phenotype_2] - lm.predict(pheno_tr[phenotype_1])

In [263]:
# check marginal 

num_SNPs = geno_tr.shape[0]
beta_list = []

print(geno_tr.shape, pheno_tr.shape, residuals.shape, lm.predict(pheno_tr[phenotype_1]).shape)
for snp_idx in range(num_SNPs):
    lm_snp = sm.OLS(endog = residuals, exog = geno_tr.iloc[snp_idx].transpose()).fit_regularized(L1_wt = 1, alpha = 1.0)
    
#     clf = Ridge(alpha = 1.0)
#     a = clf.fit(y = residuals, X = geno_tr.iloc[snp_idx].transpose())
    beta_list.append(lm_snp.params)

    if snp_idx % 1000 == 0: print(snp_idx)
        




(4887, 110) (110, 2) (110,) (110,)
0
1000
2000
3000
4000


In [264]:
beta = pd.concat(beta_list)
print(min(beta))

top_N = 100
top_N_idx = np.argsort(abs(beta))[-top_N:]

top_N_values = [beta[i] for i in top_N_idx]

print(top_N_values)

top_N_snps = geno_tr.iloc[top_N_idx].index
top_N_snps

-0.5583779953525735
[-0.07561297338427184, -0.0785144081604542, 0.07969594404724772, -0.08281263115152679, 0.08304453617046662, 0.08432435786049519, 0.08621165956338214, 0.08848861478341162, -0.09062468224959852, -0.09361998781101435, 0.09639737908915537, -0.0968898822169658, -0.09757806579644805, -0.10102446167095709, 0.10146121586962074, 0.10221283352705487, -0.10506878716451135, -0.10577592216962242, 0.11304506581670527, -0.11684903891026234, 0.1218740780184489, -0.1296356584238693, -0.1356941892652908, -0.1356941892652908, -0.1358289344836044, -0.13992001632020074, -0.1419266980019689, -0.1516560178283212, 0.15231904101351337, -0.15274495599785726, -0.1532060048431217, 0.15460657727793575, -0.1554857715221732, 0.15945614124446145, -0.1602968336594018, 0.16535375251326342, -0.16766900352553218, 0.17376565483347675, 0.1745373188081339, 0.17490001240631414, -0.1843081173721236, 0.1844525567703512, -0.18757204163971009, -0.18952304981876933, -0.20273615192443564, -0.2031264207900016, -

Index(['UNC4736200', 'UNC1143337', 'UNC2336708', 'UNC20480702', 'UNC5249903',
       'UNC24671888', 'JAX00294039', 'UNC1162577', 'JAX00259402',
       'UNC19594486', 'UNC9183513', 'UNC30427206', 'JAX00514966',
       'UNC25674790', 'UNC28305258', 'UNC21854461', 'UNC9762605',
       'UNC11667614', 'UNC9667976', 'UNC24900059', 'UNC29189342',
       'UNC26441955', 'UNC26455926', 'UNC26456769', 'UNC19594820',
       'UNC29679195', 'JAX00322333', 'UNC2372771', 'JAX00458477',
       'JAX00199468', 'JAX00322579', 'UNC4287470', 'UNC4725879', 'UNC16060158',
       'JAX00461121', 'UNC17442370', 'UNC19592744', 'JAX00688681',
       'UNC16056692', 'UNC27519627', 'UNC20435515', 'backupUNC170166828',
       'JAX00309375', 'UNC19518420', 'UNC1147792', 'UNC24887199',
       'JAX00388146', 'UNC790025', 'UNC21853099', 'UNC21853539', 'UNC30275404',
       'UNC9695930', 'JAX00102129', 'UNC30427502', 'UNC170149893',
       'backupJAX00415140', 'UNC20476358', 'UNC26408821', 'JAX00414828',
       'UNC0305569

In [265]:

predictions_re = (top_N_values * (geno_test.loc[top_N_snps].transpose())).sum(axis = 1)

In [266]:
lm_re = sm.OLS(endog = residuals, exog = geno_tr.loc[top_N_snps].transpose()).fit_regularized(L1_wt = 1.0, alpha = 1.0)

if verbose: 
    print(lm_re.params)

predictions_re2 = lm_re.predict(geno_test.loc[top_N_snps].transpose())


In [267]:
# combine the result from both
total_prediction = predictions_fe + predictions_re


mse = calculate_MSE(total_prediction, pheno_test[phenotype_2])
newmse = calculate_MSE(predictions_fe, pheno_test[phenotype_2])
newnewmse = calculate_MSE(predictions_fe + predictions_re2, pheno_test[phenotype_2])
newnewnewmse, _sample_list = top_N_snp_mixed_model_analysis(geno_df, pheno_df, phenotype_1, phenotype_2, missing_rate = missing_rate, sample_list = test_sample_list, top_N = 10)

print(mse,newmse, newnewmse, newnewnewmse)
# return(mse, test_sample_list)

The correlation between length1 and fat_mri is nan
[4555 1361 4739  948  663 4881  657 1362 2699 2091]
[0.03704802879355403, 0.03787968362660606, -0.038802188736049306, 0.038868415766717936, -0.039070866409710116, 0.040483726260727874, 0.04074132259785157, 0.04133317776789621, 0.04140515721325585, -0.04176636604985238]
Index(['UNC29338764', 'UNC9183051', 'UNC30427502', 'UNC030227936',
       'JAX00102129', 'UNC31514370', 'UNC4287470', 'UNC9183513', 'UNC17550701',
       'UNC13856737'],
      dtype='object', name='marker')
F331    1.271145
F332    0.262697
F337    0.761351
F339   -0.318885
F340    0.304734
F341   -0.393619
F342    0.558698
F345    0.220660
F346   -0.286188
F352    0.220660
F355   -0.318885
F356   -0.795981
F358   -0.488842
F359   -0.205059
F362   -0.351582
F364    0.060043
F368   -0.828678
F374   -0.521539
F377   -0.348637
F380   -0.318885
F383   -0.256436
F390    0.018006
F393    1.841588
F394   -1.103121
F395    0.528946
F396    0.220660
F398    0.761351
F405   -0.011

In [238]:
mse, test_sample_list = phenotype_correlation_analysis(geno_df, pheno_df, phenotype_1, phenotype_2, missing_rate = missing_rate, sample_list=test_sample_list)
mse


The correlation between length1 and fat_mri is nan


21.53870178358285